In [8]:
import json
import os
import argparse
import os
import numpy as np
from PIL import Image
import csv
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg

In [9]:

all_image_folders={"directinversion+p2p":"output/directinversion+p2p/annotation_images",
                       "directinversion+masactrl":"output/directinversion+masactrl/annotation_images",
                       #"directinversion+pix2pix-zero":"output/directinversion+pix2pix-zero/annotation_images",
                       "directinversion+pnp":"output/directinversion+pnp/annotation_images",
                       }
'''
all_image_folders={"ddim+p2p":"output/ddim+p2p/annotation_images",
                   "null-text-inversion+p2p":"output/null-text-inversion+p2p_3090/annotation_images",
    "negative-prompt-inversion+p2p":"output/negative-prompt-inversion+p2p/annotation_images",\
    "null-text-inversion+proximal-guidance":"output/null-text-inversion+proximal-guidance/annotation_images",
    "negative-prompt-inversion+proximal-guidance":"output/negative-prompt-inversion+proximal-guidance/annotation_images",
    "directinversion+p2p":"output/directinversion+p2p/annotation_images",
    
                       }
'''
result_path='./compare_editing'


In [10]:
def txt_draw(text,
                target_size=[512,512]):
    plt.figure(dpi=300,figsize=(1,1))
    plt.text(-0.1, -0.1, text,fontsize=3.5, wrap=True,verticalalignment="bottom",horizontalalignment="left",color='b')
    plt.axis('off')
    
    canvas = FigureCanvasAgg(plt.gcf())
    canvas.draw()
    w, h = canvas.get_width_height()
    buf = np.fromstring(canvas.tostring_argb(), dtype=np.uint8)
    buf.shape = (w, h, 4)
    buf = np.roll(buf, 3, axis=2)
    image = Image.frombytes("RGBA", (w, h), buf.tostring())
    image = image.resize(target_size,Image.Resampling.LANCZOS)
    image = np.asarray(image)[:,:,:3]
    
    plt.close('all')
    
    return image

In [12]:
src_image_folder='../dataset/annotation_images'
annotation_mapping_file='../dataset/mapping_file.json'

src_image_folders=all_image_folders

with open(os.path.join(annotation_mapping_file),"r") as f:
        annotation_file=json.load(f)


In [13]:
for key, item in annotation_file.items():

        base_image_path=item["image_path"]
        #original_prompt = item["original_prompt"].replace("[", "").replace("]", "")
        #editing_prompt = item["editing_prompt"].replace("[", "").replace("]", "")
        output_img=[]
        first=True
        for tgt_image_folder_key,image_folder in src_image_folders.items():     
            image_path=os.path.join(image_folder, base_image_path)
            src_image = Image.open(image_path)
            
            txt_img=txt_draw(tgt_image_folder_key)
            txt_img=txt_img.copy()
            
            if first:
                first=False
                output_img=np.array(src_image)
                output_img[450:,:512,:]=txt_img[450:,:,:]
            else:
                src_image=np.array(src_image)
                src_image[450:,:512,:]=txt_img[450:,:,:]
                output_img=np.concatenate((output_img,src_image),axis=0)
        
        output_path=os.path.join(result_path, base_image_path)
        dir_name=os.path.dirname(output_path)
        if not os.path.isdir(dir_name):
            os.makedirs(dir_name)
        save_img=Image.fromarray(output_img.astype('uint8'), 'RGB')
        save_img.save(output_path)
        



/tmp/ipykernel_50312/1073053126.py:10: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  buf = np.fromstring(canvas.tostring_argb(), dtype=np.uint8)
/tmp/ipykernel_50312/1073053126.py:13: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  image = Image.frombytes("RGBA", (w, h), buf.tostring())


NotImplementedError: 